# **Hands-on 1: Percepton Notebook (Tutorial)**

## **1.1 Fundamentos de la técnica**

El Perceptron es una tecnica de aprendizaje supervisado desarrollada por Rosenblatt. Este construye un clasificador lineal, es decir que por medio de una recta intenta clasificar/separar dos clases respecticamente, fucniona de manera iterativa.  
Cuando hablamos de que aprende de manera iterativa nos referimos a que con cada iteracion ajusta nuevamente la posicion de la recta mejorando su precision para que de esta manera ubicarla en la posicion en la que tenga menor margen de error.

## **1.2 Modelo Matemático del Perceptrón**

**Objetivo del modelo:** encontrar un hiperplano que separe clases linealmente separables.

**Modelo**
$$  
f(x) = \mathrm{sign}(\mathbf{w}^T \mathbf{x} + b)
$$


## **1.3 Descripción de la librería, clases, funciones (python) empleadas en programar el Perceptrón**

> *Se explicara en cada seccion cuando se vaya a utilizar*  

## **1.4 Pipeline**

### **Feature Engineering**

Se usaran todas las variables/columnas del dataset debido a que todos tienen una fuerte correlacion con la variable objetivo y no se consideran redundantes.
De igual manera las variables categoricas pasaran por una transformacion one-hot encoder para poder cuantificar las relaciones.

### **Modelo Selection**

Debido a que es un problema de clasificacion binaria (solo tenemos dos clases), debido a la baja cantiodad de parametros el uso de perceptron es totalmente posible debido a que no es muy complejo y es simple y rapido de aplicar.

### **Model Training**

Para entrenar los datos utilizaremos varias librerias.
* Pandas: Esta libreria la utilizaremos para poder cargar nuestro dataset y manipular los datos segun sea necesario.  
* skalearn: Esta libreria la utilizaremos para tratar con todo estos metodos de machine learning.  
    * OneHotEncoder: Este modulo lo utilizaremos para cuantificar las variables categoricas con sus respectivos valores.
      * ColumnTransformer: Este lo utilizaremos simplemente por comodidad de aplicar transformaciones de OneHotEncoder a todas las columnas categoricas.
    * train_test_split: lo utiliazremos para generar una seccion de entrenamiento y otra de prueba, esto para asegurarnos que nuestro modelo clasifique correctamente instancias que no se le hayan presentado.
    * Perceptron: Este es el modelo como tal, aqui se le pasaran nuestros parametros de iteraciones a realizar para posteriormente usar el metodo fit para entrenarlo.
    * accuracy_score: Con ayuda de esta biblioteca visualizaremos la puntuacion en diferentes metricas del modelo al probarlo, en este caso nos interesa el accuracy.

In [50]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

In [51]:
dataset = pd.read_csv('heart.csv')

> *Podemos observar las variables categoricas con el metodo .info, en su mayoria son las columnas con Dtype object*

In [52]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


Generamos una lista con las columnas categoricas para aplicar el metodo OneHotEncoder a todas de golpe.

In [53]:
columnas_categoricas = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina','ST_Slope']

Señalizamos cuales son las columnas que posteriormente transformaremos.

In [54]:
transformer = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), columnas_categoricas)
    ],
    remainder='passthrough'
)

Ahora generaremos nuestros conjuntos de datos:  
X -> Seran todas las variabels que nos ayudaran a clasificar nuestra variable objetivo.  
y -> Esta es una lista con nuestras variables objetivo (en este dataset es la columna HeartDisease).


In [55]:
X = dataset.drop('HeartDisease', axis=1)
y = dataset['HeartDisease']

Aplicamos las transformaciones previamente señalizadas, es decir las categoricas a numericas con OneHotEncoder.

In [56]:
X_transformed = transformer.fit_transform(X)

Generamos tanto los datos de entrenamiento como de prueba con ayuda del modulo train_test_split, utilizaremos una proporcion de 80 - 20 y un estado fijo para poder replicar este proceso varias veces y poder visualizar los resultados con facilidad.

In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y, test_size=0.2, random_state=40
)

Generaremos el modelo del perceptron, estableceremos las iteraciones.

In [58]:
clf = Perceptron(max_iter=1000, eta0=1.0, tol=1e-3)

Ahora si pasaremos a entrenar nuestro modelo con la funcion .fit

In [59]:
clf.fit(X_train, y_train)

Perceptron()

### **Prediction**

Generaremos una funcion que reciba una instancia individual (que siga el mismo formato de nuestro dataset, es decir los mismos campos) y le asigne una categoria segun nuestro modelo.

In [60]:
def clasificar(entrada):
  dataset_unico = pd.DataFrame([entrada])
  dataset_unico_transform = transformer.transform(dataset_unico)
  prediccion = clf.predict(dataset_unico_transform)[0]
  dataset_unico['HeartDisease'] = prediccion

  return prediccion, dataset_unico

Generaremos un diccionario de prueba con dichos valores.

In [61]:
nuevo_paciente = {
    "Age": 54,
    "Sex": "M",
    "ChestPainType": "ATA",
    "RestingBP": 140,
    "Cholesterol": 239,
    "FastingBS": 0,
    "RestingECG": "Normal",
    "MaxHR": 160,
    "ExerciseAngina": "N",
    "Oldpeak": 1.2,
    "ST_Slope": "Up"
}

Utilizamos la funcion y las mostramos en resultados.

In [62]:
prediccion, df_resultado = clasificar(nuevo_paciente)
print(f'La prediccion para esta instancia es: {prediccion}')
df_resultado

La prediccion para esta instancia es: 0


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,54,M,ATA,140,239,0,Normal,160,N,1.2,Up,0


### **Model Evaluation**

Utilizaremos la seccion de datos de testing que segmentamos para intentar predecirlos y posteriormente evaluarlos con su precision o accuracy.

In [63]:
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Accuracy:", acc)

Accuracy: 0.6413043478260869


Como podemos ver el modelo no tiene el mejor rendimiento en este dataset, esto indicaria que tal ves sea necesario utilizar otro modelo para este problema especifico o mejorar los parametros.

## **Referencias**

> OneHotEncoder. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html  
> User Guide — pandas 2.3.3 documentation. (s. f.). https://pandas.pydata.org/docs/user_guide/index.html  
> Perceptron. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html  
> accuracy_score. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html  
